In [2]:
import spacy 
import spacy_transformers
from spacy import displacy
import concise_concepts
import gensim.downloader as api
import json
import csv
from itertools import groupby
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [12]:
with open('train.json') as json_file:
    data = json.load(json_file)

In [5]:
nlp = spacy.load("en_core_web_trf")
model_path = "fasttext-wiki-news-subwords-300"
nlp.add_pipe("concise_concepts", config={
             "data": data, "ent_score": True
             ,"model_path": model_path
             })

key sustainability not present in word2vec model
word ecoliteracy from key sustainability not present in word2vec model
key sustainability not present in word2vec model


In [6]:
def doc_to_spans(doc):
    tokens = [(tok.text, tok.idx, tok.ent_type_) for tok in doc]
    results = []
    entities = set()
    for entity, group in groupby(tokens, key=lambda t: t[-1]):
        if not entity:
            continue
        group = list(group)
        _, start, _ = group[0]
        word, last, _ = group[-1]
        text = ' '.join(item[0] for item in group)
        end = last + len(word)
        results.append([start, end, entity])
        entities.add(entity)

    return results, entities


In [7]:
# Open CSV file and read lines into list
with open('test.csv', newline='') as f:
    reader = csv.reader(f)
    texts = [row[0] for row in reader]

# Prepare Doccano tasks in import JSONL format with the model predictions
entities = set()
tasks = []
for text in texts:
    doc = nlp(text)
    spans, ents = doc_to_spans(doc)
    entities |= ents
    tasks.append({
        'text': text,
        'label': spans
    })

In [8]:
for text in texts:
    doc = nlp(text)
    ents = doc.ents
    for ent in ents:
        new_label = f"{ent.label_}"
        ent.label_ = new_label
    doc.ents = ents

    displacy.render(doc, style="ent")

In [9]:
# Save class labels as a txt file
print('Named entities are saved to "named_entities.txt"')
with open('named_entities.txt', mode='w') as f:
    f.write('\n'.join(sorted(entities)))

Named entities are saved to "named_entities.txt"


In [10]:
# Output directory
from pathlib import Path
output_dir = Path('model/')

# Saving the model to the output directory
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = 'my_ner'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to model
